In [ ]:
import pandas as pd
import datetime
import requests
import sys


errored = []
api_key = "etherscan-api-key"
url = "https://api.etherscan.io/api"
random_state = 27

In [ ]:
list1 = pd.read_excel('data/List 1.xlsx')
list2 = pd.read_excel('data/List 2.xlsx')

In [ ]:
def wei_to_eth(wei):
    return int(wei) / 10**18

In [ ]:
def get_balance(address):
    """
    Funtion return the Eth balance of an a given address

    >>> get_balance('0x0000000000000000000000000000000000000000')
    0.00897765567

    """
    params = {
        "module": "account",
        "action": "balance",
        "address": address,
        "tag": "latest",
        "apikey": api_key
    }

    # Make a GET request with the URL and query parameters
    response = requests.get(url, params=params)

    # Print the response content
    return round(wei_to_eth(response.json()['result']),5)

In [ ]:
def get_Erc20_transaction_history(address):

    params = {
        "module": "account",
        "action": "tokentx",
        "address": address,
        "page": 1,
        "offset": 100,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": api_key
    }

    
    response = requests.get(url, params=params).json()
    return response['result']


In [ ]:
def calculate_txn_day_ratio(txn_history):
    # Convert timestamps to datetime objects and sort transactions by timestamp
    transactions = sorted(
        [datetime.datetime.fromtimestamp(int(txn['timeStamp'])) for txn in txn_history]
    )
    
    # Calculate the number of days between the earliest and latest transactions
    if len(txn_history) > 0:
        delta_days = (transactions[-1] - transactions[0]).days
    else:
        return 0
    
    # Calculate the transaction-day ratio
    if delta_days == 0:
        txn_day_ratio = len(transactions)
    else:
        txn_day_ratio = len(transactions) / delta_days
        
    return round(txn_day_ratio,3)

In [ ]:
def get_wallet_age(history: list[dict]):
    if len(history) > 0:
        creation_time = int(history[0]['timeStamp'])
        creation_date = datetime.datetime.fromtimestamp(creation_time).date()
        current_date = datetime.date.today()
        wallet_age = (current_date - creation_date).days
        return wallet_age
    else:
        return 0


In [ ]:
def to_and_from(history:list[dict],address):
    from_count = 0
    to_count = 0
    for transactions in history:
        if transactions['from'] == address:
            from_count +=1
        else:
            to_count += 1
    return from_count, to_count

In [ ]:
info = []
import time
def main(data):
    for i, address in enumerate(data['Voter']):

        if len(address) == 42:

            Erc_20_history = get_Erc20_transaction_history(address)
            
            tdr  = calculate_txn_day_ratio(Erc_20_history)
            
            eth_balance = get_balance(address)

            wallet_age = get_wallet_age(Erc_20_history)

            from_count, to_count = to_and_from(Erc_20_history,address)

            row = [address,eth_balance,tdr,wallet_age,from_count,to_count]

            info.append(row)
        
        else:
            continue
            
        # Print iteration count every 100 iterations
        if i % 5 == 0:
            print(f"Iteration: {i}")
            time.sleep(1)


In [ ]:
main(list2)

In [ ]:
columns = ['address','balance','TDR','Wallet_Age(Days)','From_Count','To_Count']
query = pd.DataFrame(info,columns=columns)

query.drop_duplicates(subset=['address'],inplace=True)
query.to_csv('list2_query.csv',index=False)